## Set up 

In [0]:
import pandas as pd
import os as os
import json as js
import gzip
import pickle
import numpy as np
import tensorflow as tf
import tflearn
import nltk
nltk.download('punkt')
from keras.utils import to_categorical
from keras.preprocessing import sequence 
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from random import shuffle

#importing libraries for NLP
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


W0817 12:04:31.420081 140702507169664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0817 12:04:31.422279 140702507169664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0817 12:04:31.439707 140702507169664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0817 12:04:31.450214 140702507169664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0817 12:04:31.472265 140702507169664 deprecation_wrapper.py:119] From /usr/local/lib/

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving df_10_08 to df_10_08


In [0]:
import pickle
with open ('df_10_08', 'rb') as fr:
    df = pickle.load(fr)

In [0]:
df.head()

,reviewerID,asin,reviewText,overall,good,Comedy,Art House & International,Drama,Action & Adventure,Horror,Science Fiction,Animation,length,!,?,",",.,,Extro,Agree,Neuro,Open,Consc,Extro_Comedy,Extro_Art House & International,Extro_Drama,Extro_Action & Adventure,Extro_Horror,Extro_Science Fiction,Extro_Animation,Agree_Comedy,Agree_Art House & International,Agree_Drama,Agree_Action & Adventure,Agree_Horror,Agree_Science Fiction,Agree_Animation,Neuro_Comedy,Neuro_Art House & International,Neuro_Drama,Neuro_Action & Adventure,Neuro_Horror,Neuro_Science Fiction,Neuro_Animation,Open_Comedy,Open_Art House & International,Open_Drama,Open_Action & Adventure,Open_Horror,Open_Science Fiction,Open_Animation,Consc_Comedy,Consc_Art House & International,Consc_Drama,Consc_Action & Adventure,Consc_Horror,Consc_Science Fiction,Consc_Animation,corpus,noun,adj,verb (simple),verb (simple past),verb (past participle),verb (gerund),adverb,preposition
0,A19JYLHD94K94D,6304239327,Jan Svankajer's feature film follow up to his ...,5.0,1,0,1,1,0,0,0,0,0.258446,-0.217939,-0.278670,0.747979,-0.732931,-1.237094,-0.126273,0.041490,-0.024634,0.560774,-0.244354,-0.0,-0.126273,-0.126273,-0.0,-0.0,-0.0,-0.0,0.0,0.041490,0.041490,0.0,0.0,0.0,0.0,-0.0,-0.024634,-0.024634,-0.0,-0.0,-0.0,-0.0,0.0,0.560774,0.560774,0.0,0.0,0.0,0.0,-0.0,-0.244354,-0.244354,-0.0,-0.0,-0.0,-0.0,jan svankaj featur film follow masterpiec alic...,0.589646,0.801383,-1.085084,-0.866250,-1.115880,0.020493,0.580656,0.141440
1,A144W50UYZG4YX,630339406X,Big-1988100mins/ColourTom HanksElizabeth Perki...,4.0,1,0,0,0,0,0,0,0,-0.011380,-0.074807,-0.278670,-0.046438,0.014425,-0.910986,-0.126273,0.053627,0.372357,0.316776,0.067297,-0.0,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,big min colourtom hankselizabeth perkinsrobert...,0.923647,1.265340,-1.373182,-0.648828,0.800366,0.674095,-0.965335,0.004659
2,A16QODENBJVUI1,B0007PALGG,Of all the films coming out this holiday movie...,4.0,1,0,0,1,0,0,0,0,2.415369,-0.217939,-0.278670,0.076878,-0.640416,-0.654365,-0.148629,-0.446772,0.372357,0.179400,0.084057,-0.0,-0.000000,-0.148629,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.446772,-0.0,-0.0,-0.0,-0.0,0.0,0.000000,0.372357,0.0,0.0,0.0,0.0,0.0,0.000000,0.179400,0.0,0.0,0.0,0.0,0.0,0.000000,0.084057,0.0,0.0,0.0,0.0,film come holiday movi season kinsey near top ...,0.789445,0.328795,-0.437283,0.246897,-0.340947,0.149816,-0.029045,1.596967
3,A3L2856DIMUXJY,B0001DCR0C,"As noted by a number of reviewers, this starte...",4.0,1,0,1,1,0,0,0,0,0.777861,-0.217939,0.014745,-0.482067,-0.167925,0.194519,-0.188938,0.037821,0.372357,0.381377,0.466380,-0.0,-0.188938,-0.188938,-0.0,-0.0,-0.0,-0.0,0.0,0.037821,0.037821,0.0,0.0,0.0,0.0,0.0,0.372357,0.372357,0.0,0.0,0.0,0.0,0.0,0.381377,0.381377,0.0,0.0,0.0,0.0,0.0,0.466380,0.466380,0.0,0.0,0.0,0.0,note number review start life play director ma...,0.210417,-0.553804,0.253153,0.446210,0.343644,0.523299,-1.193729,1.243162
4,A1Z54EM24Y40LL,B00004WI5C,This is really a cute video. It's pretty long...,4.0,1,0,0,0,0,0,0,0,-0.563680,-0.217939,-0.278670,-1.329801,-0.109748,1.545613,-0.126273,-0.215040,0.372357,0.851284,0.031202,-0.0,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,realli cute video pretti long though may hard ...,-1.756937,-0.522165,1.846531,-1.006429,-1.115880,-0.137672,1.078076,0.137840


In [0]:
df = pd.read_csv('movies_reviewers_3000.csv',index_col = 0)

## Pre-processing

In [0]:
#1 - Mapping the Labels
#Mapping labels to [0,1] where 1 is assigned to a rating greater or equal to 3(positive). Wherease, a value of 0 for a review with a rating lower than 3.
df['label'] = np.where(df.overall >= 3,1,0)


In [0]:
#2,3,4 - Uniform Spelling of words, Remove special characters, Remove stop-words

def pre_process_reviews(df):
  '''This function takes the dataframe and processing the reviews by removing the punctuation, stopwords and making
  all the reviews into lowercase so that all reveiews have an identical structure. It then returns a clean corpus 
  consisting of all the processed reviews.'''
  
  corpus = []
  
  for i in range(len(df)):
    #Getting rid of punctuation
    review = re.sub('[^a-zA-Z]', ' ', str(df.iloc[i]['reviewText']))
    #Making everything lowercase
    review = review.lower()
    #splitting the sentence into words
    review = review.split()
    #delete stop words 
    review = [word for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
    
  return corpus

corpus = pre_process_reviews(df)

In [0]:
print('This is what our reviews look like after removing all the special characters and stopwords:')
corpus[:3]

This is what our reviews look like after removing all the special characters and stopwords:


['jan svankajer feature film follow masterpiece alice rather free interpretation classic often referenced tale faust like alice svankmajer approaches story unique original fashion conceptually visually usual mixes various genres art forms combining live actors puppetry claymotion faust sometimes ordinary man playing role faust sometimes puppet sometimes faust film shows svankmajer brilliant creativity imagination fantastic unique stop motion technique best even though find slightly inferior alice overdubbed voices annoying much better alice wonderful surreal dark atmosphere brilliant character designs even couple scenes quite entertaining like old man collects legs adore way mephistopheles turns clay mirror faust head changing puppet devil angel delicate transformation faust turning man puppet back nothing else visual side faust well worth seeing fan svankmajer interested classic animation puppetry buy faust',
 'big mins colourtom hankselizabeth perkinsrobert loggiajohn heardthe wonder

In [0]:
#5 - Building the vocabulary 

def vocabulary(corpus):
  '''This function takes the entire corpus of the reviews and returns a list of all the unique words that it contains 
  as the vocabulary for our dataset.'''
  
  revs = ''.join(corpus)
  words = revs.split(' ')
  vocab = set(words)
  
  return list(vocab)
 
vocab = vocabulary(corpus)
vocab[:10]

['tamilnadu',
 'virgins',
 'emcompetent',
 'tchicovasky',
 'certifiable',
 'washy',
 'bodiam',
 'surviors',
 'downfall',
 'beseiged']

In [0]:
#6 - Word-to-Index

char_to_idx = {word: i for i, word in enumerate(vocab)}


def revs_to_idx(corpus):
  '''This function takes the corpus of all the reviews, converts the sequence of words to a sequence of integers 
  and return a corpus of all the reviews in the form of integers '''
  
  final = []

  for i in range(len(corpus)):
    review = corpus[i]
    words = review.split(' ')
    rev = []
    for j in range(len(words)):
      try:
        rev.append(char_to_idx[words[j]])
      except KeyError:
        rev.append(0) #unknown word - that is, we are going to replace a word with 0 if it not contained in the vocabulary
    final.append(rev)
    
  return final

inds_reviews = revs_to_idx(corpus)
print(inds_reviews[0])

[11936, 56879, 19879, 81163, 76060, 54574, 50245, 36325, 53721, 15889, 81795, 23092, 69992, 51492, 19354, 67199, 50245, 54751, 13013, 4744, 73784, 78005, 40085, 39366, 30568, 23911, 39843, 63845, 43989, 40488, 76092, 19345, 77595, 23271, 70495, 63652, 19354, 71370, 59604, 68887, 64980, 51380, 19354, 71370, 35843, 71370, 19354, 81163, 18092, 54751, 12590, 3429, 29632, 54262, 73784, 74257, 62280, 38037, 64911, 5155, 35749, 655, 8201, 69426, 50245, 68337, 61218, 80763, 3447, 1069, 50245, 67777, 63040, 48588, 78996, 12590, 39514, 15518, 5155, 24517, 64277, 45375, 33059, 67199, 70785, 68887, 9492, 11303, 9612, 48869, 69731, 19737, 61113, 48756, 19354, 39603, 81806, 35843, 907, 81294, 70247, 79106, 19354, 22583, 68887, 35843, 48446, 39936, 72907, 17053, 32872, 19354, 14611, 62087, 38233, 45467, 54751, 53615, 81795, 69906, 70495, 62153, 19354]


In [0]:
inds_reviews = revs_to_idx(corpus)
print(df[5:6]['reviewText'],'was transformed to :', inds_reviews[5:6])


5    A truly "kids" movie.  Kids are the stars, and...
Name: reviewText, dtype: object was transformed to : [[33756, 42265, 23705, 42265, 16739, 73634, 54517, 39717, 6902, 25509, 1617, 79958, 73515, 57105, 58590, 76714, 30374, 6391]]


In [0]:
df.iloc[5][2]

'A truly "kids" movie.  Kids are the stars, and the aliens.  What a terriffic twist to the end.  Great Sci-fi in outer space adventure and fun from start to finish.'

In [0]:
#7 - Padding

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence 
max_review_length = 250

def padding(inds,max_review_length = 250):
  data = sequence.pad_sequences(inds, maxlen=max_review_length)
  return data
review_data = padding(inds_reviews,max_review_length)# sequence.pad_sequences(inds_reviews, maxlen=max_review_length)
labels = df['label'].values
#labels = to_categorical(labels,2)

## Building the model

In [0]:
#Splitting the data

trainX, testX, trainY, testY = train_test_split(review_data,labels,test_size = 0.20, random_state = 1) 
pd.DataFrame(trainX).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63940,56176,67878,45454,10029,63767,250,45454,53850,3988,25621,40782,66269,36545,61843,62977,22620,29613,71860,73919
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36207,24683,36207,45688,54460,56219,72598,48809,30658,23206,38397,5233,7176,25541,36207,32370,74254,58004,16838,41005
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,47513,59890,53263,53884,71616,53171,20931,16487,20285,63123,5446,38814,37602,6283,63112,10061,59890,53263,3521,45418,66555,7663,54230,7775,77612,77612,75706,80423,3042,59890,53263,65059,76258,4013,74829,79100,74829,57518,36495,27787
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,40782,32578,71328,49593,51838,74829,2909,74202,67268,58000,24740,61305,42978,48809,46748,80116,53783,57837,36195,44025,5754,60453,7027,5100,43979,13581,10196,927,38715,608,66250,3024,63940,3016,36027,74160,15148,59912,63905,34914
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7137,1998,70186,58063,50925,76258,58063,44879,64460,40347,3521


In [0]:
df.columns

Index(['reviewerID', 'asin', 'reviewText', 'overall', 'good', 'Comedy',
       'Art House & International', 'Drama', 'Action & Adventure', 'Horror',
       'Science Fiction', 'Animation', 'length', '!', '?', ',', '.', ' ',
       'Extro', 'Agree', 'Neuro', 'Open', 'Consc', 'Extro_Comedy',
       'Extro_Art House & International', 'Extro_Drama',
       'Extro_Action & Adventure', 'Extro_Horror', 'Extro_Science Fiction',
       'Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International',
       'Agree_Drama', 'Agree_Action & Adventure', 'Agree_Horror',
       'Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',
       'Neuro_Art House & International', 'Neuro_Drama',
       'Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International',
       'Open_Drama', 'Open_Action & Adventure', 'Open_Horror',
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy',
       'Consc_Art House & Inter

In [0]:
from keras.models import Model, Input
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras import layers, models

nlp_input = Input(shape=(250,), name='nlp_input')
meta_input = Input(shape=(62,), name='meta_input')
emb = Embedding(output_dim=64, input_length=250, input_dim = 250)(nlp_input)
nlp_out = Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01)))(emb)
x = concatenate([nlp_out, meta_input])
x = Dense(1, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[nlp_input , meta_input], outputs=[x])

In [0]:
#Building a model 
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout


embedding_vecor_length = 64
model = Sequential() 
model.add(Embedding(len(vocab), embedding_vecor_length, input_length=max_review_length)) 
#model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')) 
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')) 
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')) 
model.add(MaxPooling1D(pool_size=2)) 
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary()) 


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 250, 64)           5247744   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 250, 64)           12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 125, 64)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)               212000    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 201       
Total params: 5,472,297
Trainable params: 5,472,297
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Training the model 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
#Evaluating the test data
#history = model.fit([trainX, trainO], trainY, epochs = 25, batch_size=64, shuffle = True, verbose = 1 ) #64 reviews at one time #will see the dataset 3 times
history = model.fit(trainX, trainY, epochs = 25, batch_size=64, shuffle = True, verbose = 1 )
_, test_ac = model.evaluate(testX, testY, verbose=1)

test_acc = []

for i in range(len(history.history['acc'])):
  test_acc.append(history.history['acc'][i])

test_loss = []

for i in range(len(history.history['acc'])):
  test_loss.append(1 - history.history['acc'][i])
  
print('Test Accuracy: {}'.format(test_ac))


Epoch 1/25
15046/15046 [==============================] - 127s 8ms/step - loss: 0.4446 - acc: 0.7790
Epoch 2/25
15046/15046 [==============================] - 122s 8ms/step - loss: 0.1913 - acc: 0.9268
Epoch 3/25
15046/15046 [==============================] - 123s 8ms/step - loss: 0.0889 - acc: 0.9705
Epoch 4/25
15046/15046 [==============================] - 123s 8ms/step - loss: 0.0428 - acc: 0.9864
Epoch 5/25
15046/15046 [==============================] - 123s 8ms/step - loss: 0.0266 - acc: 0.9910
Epoch 6/25
15046/15046 [==============================] - 122s 8ms/step - loss: 0.0269 - acc: 0.9912
Epoch 7/25
15046/15046 [==============================] - 123s 8ms/step - loss: 0.0196 - acc: 0.9942
Epoch 8/25
15046/15046 [==============================] - 125s 8ms/step - loss: 0.0111 - acc: 0.9964
Epoch 9/25
15046/15046 [==============================] - 124s 8ms/step - loss: 0.0072 - acc: 0.9981
Epoch 10/25
15046/15046 [==============================] - 125s 8ms/step - loss: 0.0103 - a